In [1]:
import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [25, 15]
plt.rcParams.update({'font.size': 25})

In [4]:
def load_dataframe_from_files(dirin, fileprefix):
    import glob
    files = glob.glob(os.path.join(dirin, fileprefix))
    print("[Info] Loading {}".format(files[0]))
    df = pd.read_csv(files[0], comment='#', index_col=False)
    for file in files[1:]:
        print("[Info] Loading {}".format(file))
        dftmp = pd.read_csv(file, comment='#', index_col=False)
        df = pd.concat([df, dftmp])
    return df

In [8]:
dirin = os.path.join("..", "..", "Data", "MuonsROI", "06-14-2020-QuantumEff")
filepref = "roi_RndSeed123456789_part1_part*csv"
df = load_dataframe_from_files(dirin, filepref)
df = df[df.columns[:-1]]

[Info] Loading ../../Data/MuonsROI/06-14-2020-QuantumEff/roi_RndSeed123456789_part1_part1.csv
[Info] Loading ../../Data/MuonsROI/06-14-2020-QuantumEff/roi_RndSeed123456789_part1_part2.csv
[Info] Loading ../../Data/MuonsROI/06-14-2020-QuantumEff/roi_RndSeed123456789_part1_part3.csv
[Info] Loading ../../Data/MuonsROI/06-14-2020-QuantumEff/roi_RndSeed123456789_part1_part4.csv
[Info] Loading ../../Data/MuonsROI/06-14-2020-QuantumEff/roi_RndSeed123456789_part1_part5.csv
[Info] Loading ../../Data/MuonsROI/06-14-2020-QuantumEff/roi_RndSeed123456789_part1_part6.csv
[Info] Loading ../../Data/MuonsROI/06-14-2020-QuantumEff/roi_RndSeed123456789_part1_part7.csv


In [9]:
op_yield = 40    # 40 OP / KeV
quantum_eff = 0.40
df["pedetected"] = np.ceil(df.energydeposition * op_yield * quantum_eff * df.detectionefficiency)    # round to next int
df["energydetected"] = df.pedetected / op_yield

In [41]:
edep_gby_event = df.groupby("eventnumber").energydeposition.sum()
nr_events_20mev = len(edep_gby_event[edep_gby_event<20000])
nr_events_zero = len(edep_gby_event[edep_gby_event==0])
nr_events = len(edep_gby_event)
print("Considering ALL Depositions in ROI:")
print("Number of events: {}".format(nr_events))
print("Number of events <20MeV: {} ({:.2f}%)".format(nr_events_20mev, nr_events_20mev/nr_events*100))
print("Number of events <=0MeV: {} ({:.2f}%)".format(nr_events_zero, nr_events_zero/nr_events*100))

Considering ALL Depositions in ROI:
Number of events: 20751
Number of events <20MeV: 8264 (39.82%)
Number of events <=0MeV: 886 (4.27%)


In [45]:
nonzero_edep_df = df[df.energydeposition>0]
edep_gby_event = nonzero_edep_df.groupby("eventnumber").energydeposition.sum()
nr_events_20mev = len(edep_gby_event[edep_gby_event<20000])
nr_events = len(edep_gby_event)
print("Considering Non-Zero Depositions in ROI:")
print("Number of events: {}".format(nr_events))
print("Number of events <20MeV: {} ({:.2f}%)".format(nr_events_20mev, nr_events_20mev/nr_events*100))

Considering Non-Zero Depositions in ROI:
Number of events: 19865
Number of events <20MeV: 7378 (37.14%)


In [ ]:
plt.hist(edep_gby_event/1000, bins=1524460, cumulative=True, density=True, color='w', edgecolor='b')
plt.xlim(0, 100)
plt.ylim(0, 0.50)
plt.vlines(20, ymin=0, ymax=0.50, color='red')
plt.title("Integral Curve of Edep in LAr - [0,100] MeV")
plt.ylabel("Fraction of events below this energy")
plt.xlabel("Total LAr energy deposition (MeV)")
plt.show()